<a href="https://colab.research.google.com/github/Alessandro-Borges/alessandro-borges.github.io/blob/main/dev_equipe_MVP_bolsas_prouni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Software de Alocação Ótima de Bolsas

# Estruturado em quatro componentes :
#1 - Input de Dados,
#2 - Modelagem e ETL dos dados para formato algebrico de otimização e calculos necessários,
#3 - Otimização matemática conforme parâmetros selecionados,
#4 - Output do resulato otimo em tabelas e dashboard/relatório

In [ ]:
# Importando bibliotecas necessárias
!pip install pyomo

In [ ]:
# Instala o solver gratuito - GLPK

!apt-get install -y -qq glpk-utils

In [ ]:
# Importando bibliotecas necessárias

import tkinter  as tk
from tkinter import *
import pyomo.environ as pyo
import pandas as pd
import numpy as np
import os
from ipywidgets import widgets, interact, Button, VBox
from google.colab import files


# Nova seção

# 1 INPUT DE DADOS

In [ ]:
#https://stackoverflow.com/questions/50479576/google-colab-changing-directory

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 2 Transformação dos dados

## 2.1 Transformação dos dados de entrada para formulação do modelo

In [ ]:
os.path.abspath(".")
CURRENT_DIR = os.path.abspath(".")


In [ ]:
dados = {'ID_BOLSA': [1, 2, 3], 'Descricao': ['Prouni100','Prouni50','SemBolsa']}
df_bolsas = pd.DataFrame(data=dados)

dados_periodos = {'ID_PERIODO': [1, 2], 'Descricao': ['Calouro','Veterano']}
df_periodos = pd.DataFrame(data=dados_periodos)

df_cursos =  pd.read_csv(os.path.join(CURRENT_DIR,'Lista_Cursos.csv'), delimiter =";")
df_alunos_matriculados_novo =  pd.read_csv(os.path.join(CURRENT_DIR,'Alunos_Matriculados_Teste.csv'), delimiter =";")
df_receitas_novo =  pd.read_csv(os.path.join(CURRENT_DIR,'Receitas_teste_padronizada.csv'), delimiter =";")


## 2.2 - Modelagem e ETL dos dados para formato algebrico de otimização e calculos necessários,

In [ ]:
df_alunos_matriculados_novo = df_alunos_matriculados_novo.pivot(index = 'CR', columns = 'PERIODO', values ='N_ALUNOS').reset_index() # faz o pivot e joga o Periodo pra coluna pra cada CR
df_alunos_matriculados_novo = df_alunos_matriculados_novo.to_numpy()
df_alunos_matriculados_novo = np.delete(df_alunos_matriculados_novo,0,axis=1) # exclui a 1 coluna e tira o CR

In [ ]:
iix_n = pd.MultiIndex.from_product([np.unique(df_receitas_novo.CR), np.unique(df_receitas_novo.Periodo)])
receitas = (df_receitas_novo.pivot_table('TKT_MEDIO_PERIODO', ['CR', 'Periodo'], 'ID_BOLSA', aggfunc='first')
         .reindex(iix_n).to_numpy()
         .reshape(df_receitas_novo.CR.nunique(),df_receitas_novo.Periodo.nunique(),-1))

# 3 MODELO PLI e OTIMIZAÇÃO MATEMÁTICA

In [ ]:
# 3 Modelo PLIM e Otimização Matemática

#Declaração do modelo:

modelo = pyo.ConcreteModel()

# Indices
cursos = df_cursos['CR'] # Indice J
bolsas = df_bolsas['ID_BOLSA'] # Indice K
periodos = df_periodos['ID_PERIODO'] # Indice I

m = len(cursos) # Linhas das matrizes, total de cursos
n = len(periodos) # colunas das matrizes, total de periodos, neste caso, considerando calouros/ periodo 1 e veteranos periodo 2, pois só entrariam bolsistas no periodo 1
o = len (bolsas) # nº de matrizes (m x n) a serem criadas , 3  k = 1 ou 2 ou 3


modelo.J = pyo.RangeSet(m) # Cursos --  Indice J
modelo.I = pyo.RangeSet(n) # Periodos --  Indice I
modelo.K = pyo.RangeSet(o) # Bolsas -- Indice K

## 3.1 PARÂMETROS

In [ ]:
############# PARAMETROS ######

# Caso ocorra algum erro, ele irá acusar que já existe o parametro ou índice no modelo criado, ai por isso, talvez seja necessário deletar o índice ou o param do modelo

#del modelo.alunos_index  # Delete the component if it already exists

In [ ]:
modelo.cursos = pyo.Param(modelo.J, initialize=lambda modelo, j: cursos[j-1])
modelo.periodos = pyo.Param(modelo.I, initialize=lambda modelo, i: periodos[i-1])
modelo.bolsas = pyo.Param(modelo.K, initialize=lambda modelo, k: bolsas[k-1])
modelo.receitas = pyo.Param(modelo.J, modelo.I, modelo.K, initialize=lambda modelo, j, i, k: receitas[j-1][i-1][k-1]) #revisitar formato que tem que ficar
modelo.alunos = pyo.Param(modelo.J, modelo.I, initialize=lambda modelo, j, i: df_alunos_matriculados_novo[j-1][i-1])



## 3.2 Variáveis de decisão

In [ ]:
# Variáveis de decisão:

modelo.y = pyo.Var(modelo.J, modelo.I, modelo.K, within = pyo.PositiveReals)

## 3.3 Função Objetivo

In [ ]:
# Função Objetivo

def regra_z(mod):
    return (pyo.summation(mod.receitas, mod.y))

modelo.z = pyo.Objective(rule=regra_z, sense=pyo.maximize)

In [ ]:
#modelo.y.pprint()

In [ ]:
#modelo.z.pprint()

## 3.4 RESTRIÇÕES

In [ ]:
# RESTRICOES

# RESTRIÇÃO 1 : cada periodo de cada curso precisa ter pelo menos 10% de seus alunos bolsistas integrais

# Uma forma de fazer a mesma restrição que você fez por lambda é esta:
def f_restr1(modelo, j, i, k):
    if k == 1:
        return modelo.y[j,i,k] >= round(0.1*modelo.alunos[j,i],0)
    else:
        return pyo.Constraint.Skip

modelo.restr_1 = pyo.Constraint(modelo.J, modelo.I, modelo.K, rule=f_restr1)


# Restrição 2: Estabelece que o total de alunos bolsistas na universidade deve
# perfazer pelo menos 20% do total de estudantes pagantes da universidade. Alunos pagantes são
# todos os alunos que não têm bolsa integral.
# Alunos com bolsa parcial 50%, podem ser contabilizados que a cada 2 deles, seria um integral, por isso coef "0,5" na variável "B"

def f_restr2(modelo):
    A = sum(modelo.y[j,i,1] for j in modelo.J for i in modelo.I) # não precisa usar os índices K1 / K2 / K3
    B = sum(modelo.y[j,i,2]*0.5 for j in modelo.J for i in modelo.I) # só passar o valor de k de forma explícita
    C = sum(modelo.y[j,i,3]*-0.201 for j in modelo.J for i in modelo.I) # no índice da variável y
    return (A + B + C) >= 0

modelo.restr_2 = pyo.Constraint(rule=f_restr2)


# Restrição 3 : O nº de alunos em cada periodo i de cada curso j tem que ser igual ao
#total de alunos cursando aquele periodo daquele curso j para todos os tipos de bolsa

def f_restr3(modelo, j, i):
    return sum(modelo.y[j,i,k] for k in modelo.K) == round(modelo.alunos[j,i],0)


modelo.restr_3 = pyo.Constraint(modelo.J, modelo.I, rule = f_restr3)


# RESTRICAO 4, NUMERO ALUNOS BOLSISTAS 100% DEVE SER = OU > QUE NUMERO ATUAL
# RESTRICAO 4, NUMERO ALUNOS BOLSISTAS 50% DEVE SER = OU > QUE NUMERO ATUAL

# RESTRICAO 5, NUMERO ALUNOS BOLSISTA 100% NAO PODE SER SUPERIOR QUE NUMERO X, DETERMINADO POR OUTRAS REGRAS (LIMITE SUPERIOR DE BOLSAS)
# RESTRICAO 5, NUMERO ALUNOS BOLSISTA 50% NAO PODE SER SUPERIOR QUE NUMERO X, DETERMINADO POR OUTRAS REGRAS (LIMITE SUPERIOR DE BOLSAS)




## 3.5 Resolução do modelo

In [ ]:
resultado = pyo.SolverFactory('glpk',executable = '/usr/bin/glpsol').solve(modelo)

modelo.z()

43871946.177119985

# 4 ANÁLISE DOS RESULTADOS

## 4.1 Captura dos resultados do modelo

In [ ]:
#https://stackoverflow.com/questions/67491499/how-to-extract-indexed-variable-information-in-pyomo-model-and-build-pandas-data

#https://or.stackexchange.com/questions/6122/how-to-display-results-of-pyomo-variables-in-a-pandas-dataframe

# let's convert each var to a pandas series, indexed by model.T...

# get all the variables (assuming the fuller model will have constraints, params, etc.)
model_vars = modelo.component_map(ctype=pyo.Var)


serieses = []   # collection to hold the converted "serieses"
for k in model_vars.keys():   # this is a map of {name:pyo.Var}
    v = model_vars[k]
    #print(v)

    # make a pd.Series from each
    s = pd.Series(v.extract_values(), index=v.extract_values().keys())
    #print(s)

    # if the series is multi-indexed we need to unstack it...
    if type(s.index[0]) == tuple:  # it is multi-indexed
        s = s.unstack(level=1)
    else:
        s = pd.DataFrame(s)         # force transition from Series -> df

    #print(s)

    # multi-index the columns
    s.columns = pd.MultiIndex.from_tuples([(k, t) for t in s.columns])

    serieses.append(s)

df = pd.concat(serieses, axis=1)

# Reset the index to make the new columns separate from the MultiIndex
df.reset_index(inplace=True)

df.rename(columns={'level_0': 'Curso', 'level_1': 'tipo_bolsa'}, inplace=True)

# Flatten the MultiIndex columns and rename the columns

#f'{col[0]}_{col[1]}' if col[0] == 'y' else col[0]:
#This is a conditional expression (also known as a ternary operator).
# It checks if the first level of the MultiIndex for the current column (col[0]) is equal to 'y'.
#If it is, it creates a new column name by concatenating the first level (col[0])
#with an underscore _ and the second level (col[1]).
# If the condition is not met (i.e., the first level is not 'y'), it keeps the original column name (col[0]).

#The entire expression is wrapped in square brackets ([]),
# which is a list comprehension. This generates a new list of column names based on the conditions specified.

#The f'{col[0]}_{col[1]}' is an f-string, a formatted string literal in Python.
#It allows you to embed expressions inside string literals, using curly braces {}.
#In this case, it's used to concatenate the first level (col[0]) with an underscore _
#and the second level (col[1]) to form a new column name.

#df.columns = [f'{col[0]}_{col[1]}' if col[0] == 'y' else col[0] for col in df.columns]

df.columns = [f'{col[1]}' if col[0] == 'y' else col[0] for col in df.columns]


#df['y_3'] = df['y_1'] + df['y_2']

#print(df)

In [ ]:
# Unpivot or melt the DataFrame
df = pd.melt(df, id_vars=['Curso', 'tipo_bolsa'], var_name='PERIODO', value_name='N_STUDENTS')

In [ ]:
df['Curso'] = df['Curso'].astype(int)
df['PERIODO'] = df['PERIODO'].astype(int)

In [ ]:
print(df)

## 4.2 Cálculo das métricas/KPI's necessários

In [ ]:
#Calcular as seguintes metricas e fazer um "dash" :

# Total de Estudantes -- OK
# Nº de Estudantes sem bolsa -- OK


# Nº de bolsistas 100% -- OK
# Receita "perdida" em R$ (mensal) com bolsas 100% -- OK

# Nº de bolsistas 50% -- OK
# Receita "perdida" em R$ (mensal) com bolsas 50% -- OK

# Receita Total Esperada "Líquida" ( bolsa tipo 3 + bolsa tipo 2) ou total sum bolsas - dedução de bolsa tipo 1 -- OK

# n ingressante bolsa 100% (total alunos antes da otimizacao bolsa 100 - total aluno bolsa 100 apos otimizacao), tipo bolsa = 1
# n ingressante bolsa 50% (total alunos antes da otimizacao bolsa 50 - total aluno bolsa 50 apos otimizacao), tipo bolsa = 2

# Nº Total de Bolsistas

# Nº de bolsistas ponderado 100%

# Valor gasto com bolsas no semestre

# Nº de alunos pagantes


## -- metricas acima e talvez outras calculadas (exemplo abaixo) e exibidas em uma tabal resumo, 1 valor de cada coluna.
## -- provavelmente será necessário pivot na tabela para criar como colunas adicionais os valores normalizar/desnormalizar.
#abaixo calculas e disponivel em uma tabela , exibindo por curso

# 8 tabela com Nº total aluno calouros, veteranos, total, nº bolsa 100% , Nº bolsa 50%

# 9 Valor gasto em R$ (mensal/semestral) com bolsas 100%,
# 10 Valor gasto em R$ (mensal/semestral) com bolsas 50%,
# 11 Valor Total Gasto em r$ (mensal/semestral) com bolsas

## -- valores disponiveis para consulta em uma tabela , exibindo por curso

# tkt med calouro
# tkt med veteranos
# total alunos
# total calouros (periodo = 1)
# total alunos veteranos (periodo = 2)
# n ingressante bolsa 100% (total alunos antes da otimizacao bolsa 100 - total aluno bolsa 100 apos otimizacao), tipo bolsa = 1
# n ingressante bolsa 50% (total alunos antes da otimizacao bolsa 50 - total aluno bolsa 50 apos otimizacao), tipo bolsa = 2

### 4.2.1 Obtencao dos tkt medio por curso e periodo

In [ ]:
# Cópia do dataframe de receitas informado pelo usuario

df_receitas_novo_teste2 = df_receitas_novo.copy()

# Filtro para pegar os valores integrais de tkt medio de cada curso, para periodo = 1, calouro e periodo = 2 veterano

df_receitas_novo_teste2 = df_receitas_novo_teste2[df_receitas_novo_teste2['ID_BOLSA'] == 3]

# Create a dictionary to map unique 'CR' values to unique IDs
cr_id_mapping = {cr: idx + 1 for idx, cr in enumerate(df_receitas_novo_teste2['CR'].unique())}

# Add an "id" column based on the mapping using .loc
df_receitas_novo_teste2.loc[:, 'id'] = df_receitas_novo_teste2['CR'].map(cr_id_mapping)

### --- Maneira "menos performatica" ou que pode ter comportamentos inesperados, baseando no map
# Add an "id" column based on the mapping
#df_receitas_novo_teste['id'] = df_receitas_novo_teste['CR'].map(cr_id_mapping)

#value is trying to be set on a copy of a slice from a DataFrame.
#Try using .loc[row_indexer,col_indexer] = value instead

#See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Check column types
#column_types = df_receitas_novo2.dtypes



### 4.2.2 Cálculo da receita associada a alocação de bolsas

In [ ]:
# Cópia do dataframe de resultados após otimização do modelo

df2 = df.copy()

# Merge DataFrames de resultados x tkt medio
merged_df = pd.merge(df_receitas_novo_teste2, df2, left_on=['id','Periodo'], right_on=['Curso','PERIODO'], how='inner')

# Cálculo do valor de receita/bolsa associada a cada curso e numero de estudantes após otimização:

#axis parameter in the apply lambda function :
#axis=0 means apply the function to each column independently.
#axis=1 means apply the function to each row independently.
#axis=0: Apply the function along rows (vertically). This means that the function would be applied to each column independently.
#axis=1: Apply the function along columns (horizontally). This means that the function would be applied to each row independently.

#In your case, with the given lambda function and assignment, you should keep axis=1 to ensure the correct calculation for each row.

merged_df['Valor_Bolsa'] = merged_df.apply(lambda x: x['N_STUDENTS'] * x['TKT_MEDIO_PERIODO'], axis=1)


In [ ]:
merged_df_bolsa100 = merged_df[merged_df['tipo_bolsa'] == 1]
merged_df_bolsa50 = merged_df[merged_df['tipo_bolsa'] == 2]
merged_df_sembolsa = merged_df[merged_df['tipo_bolsa'] == 3]

In [ ]:
kpi_bolsa_100 = merged_df_bolsa100.groupby(['Curso', 'PERIODO']).agg({'Valor_Bolsa': 'sum','N_STUDENTS': 'sum'}).reset_index()
kpi_bolsa_50 = merged_df_bolsa50.groupby(['Curso', 'PERIODO']).agg({'Valor_Bolsa': 'sum','N_STUDENTS': 'sum'}).reset_index()
kpi_bolsa_sembolsa = merged_df_sembolsa.groupby(['Curso', 'PERIODO']).agg({'Valor_Bolsa': 'sum','N_STUDENTS': 'sum'}).reset_index()

#df_results_kpi['Valor_Bolsas_Mensal_100'] = merged_df[merged_df['tipo_bolsa'] == 1].groupby(['Curso','PERIODO']).agg({'Valor_Bolsa': 'sum'})
#df_results_kpi['Valor_Bolsas_Mensal_100'] = merged_df[merged_df['tipo_bolsa'] == 1].groupby(['Curso', 'PERIODO']).agg({'Valor_Bolsa': 'sum'})


In [ ]:
#CALCULO DE KPIS

#Set the display option for float values
pd.set_option('display.float_format', '{:.2f}'.format)

#KPIS - RECEITA

# Receita Total Bruta
kpi_total_ROB = merged_df.agg({'Valor_Bolsa': 'sum'})
kpi_total_ROB = round(kpi_total_ROB, 2)

# Receita Bolsa_100
kpi_total_bolsa_100 = merged_df_bolsa100.agg({'Valor_Bolsa': 'sum'})
kpi_total_bolsa_100 = round(kpi_total_bolsa_100, 2)

# Receita Total Líquida
kpi_total_ROL = merged_df.agg({'Valor_Bolsa': 'sum'}) - merged_df_bolsa100.agg({'Valor_Bolsa': 'sum'})
kpi_total_ROL = round(kpi_total_ROL, 2)

#KPIS - ESTUDANTES

# Total de estudantes na instituição
kpi_total_estudantes = merged_df.agg({'N_STUDENTS': 'sum'})
rounded_kpi_total_estudantes = round(kpi_total_estudantes, 2)

# Total de estudantes bolsa integral PROUNI 100%
kpi_total_estudantes_bolsa100 = merged_df_bolsa100.agg({'N_STUDENTS': 'sum'})
rounded_kpi_total_estudantes_bolsa100 = round(kpi_total_estudantes_bolsa100, 2)


# Total de estudantes bolsa parcial PROUNI 50%
kpi_total_estudantes_bolsa50 = merged_df_bolsa50.agg({'N_STUDENTS': 'sum'})
rounded_kpi_total_estudantes_bolsa50 = round(kpi_total_estudantes_bolsa50, 2)


# Total de estudantes sem bolsa
kpi_total_estudantes_sembolsa = merged_df_sembolsa.agg({'N_STUDENTS': 'sum'})
rounded_kpi_total_estudantes_sembolsa = round(kpi_total_estudantes_sembolsa, 2)

